## How to convert shapefile to WKT.

Need to Convert a shapefile to WKT so we can use it as a Geometry field in our WaDE imports.

In [1]:
# #Needed Libararies
# import os
# import numpy as np
# import pandas as pd
# import geopandas as gpd # the library that lets us read in shapefiles
# pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
# #Working Directory and Input Files
# workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/USBR/Colorado River Basin Natural Flow and Salt Data/Summary Sheet/AggregatedAmounts/RawInputData/byState"
# os.chdir(workingDir)

# USBR_Tri = gpd.read_file('C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/USBR/Colorado River Basin Natural Flow and Salt Data/Summary Sheet/AggregatedAmounts/RawInputData/byState/ByState.shp')

In [3]:
# USBR_Tri.head(20)

In [4]:
# df = pd.DataFrame(USBR_Tri)
# df

In [5]:
# # Printing file to xlsx
# df.to_excel('USBR_Tri.xlsx', index=False)  # The output

# Working with USBR Aggregated Data

Pre-processing input data for a smoother upload experience of the state data to the WaDE 2.0 database.
Using geopandas to read in shp file, and coverting to WKT for ReportingUnit geometry.

#### Notes:
- NA

In [6]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [7]:
#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/USBR/Colorado River Basin Natural Flow and Salt Data/Summary Sheet/AggregatedAmounts/RawInputData"
os.chdir(workingDir)
fileInput = "perfect example_A.csv"
df = pd.read_csv(fileInput)
df.head(3)

,USE TYPE,STATE,COUNTY,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Irrigated Agriculture,Arizona,san juan-colorado river,2500.0000,2900.0,4000.0,4250.0,3540.0,2690.0,3810.0,4300.0,5320.0,6290.0,4575.000,5664.000,6523.000,5800.000,5800.000,3838.9,3377.1,3275.1,3371.76,3620.0,4.500000e+03,3790.000,4520.0000,4.900000e+03,4300.0000,4.259000e+02,398.8000,463.3000,485.8000,6.494000e+02,515.0,436.3,487.9,580.4,609.4,572.3,958.2128,1081.5,729.5776,1209.563,1014.4311,6.128336e+02,1055.6067,968.0913,1178.6036,1.114712e+03,1.106770e+03,1.019725e+03
1,Irrigated Agriculture,Colorado,green river,147229.0000,128045.0,120533.0,140336.0,124835.0,117430.0,101285.0,135420.0,134259.0,129310.0,156092.888,142308.266,139679.482,136239.364,131406.655,120153.9,137987.2,185849.9,167111.60,169852.1,1.680848e+05,190939.854,137093.9329,1.517597e+05,107409.5306,1.918523e+05,131402.0834,162343.0776,150787.5710,1.904855e+05,177121.0,215140.0,180779.1,131425.3,118595.7,148442.5,155904.6000,151792.8,204694.9000,216359.600,198029.0000,2.764474e+05,181391.0000,154112.0000,169262.5000,1.723303e+05,1.648434e+05,2.012125e+05
2,Irrigated Agriculture,Colorado,upper main stem,985970.5317,974215.0,842480.0,1013597.0,921269.0,904678.0,822262.0,986244.0,984831.0,1006678.0,1059416.541,1016864.749,1009453.272,922455.222,1011258.749,836124.4,964544.0,1089711.2,1125820.50,1033206.6,1.079455e+06,1103990.338,912638.6260,1.111273e+06,762836.0924,1.013847e+06,835403.0995,964411.3680,850087.7871,1.080423e+06,1077097.5,1006916.2,947432.0,874501.4,807957.1,795027.2,836905.4000,906965.4,885541.0000,883359.400,889468.2000,1.116340e+06,802324.7000,715887.5000,665396.3000,1.014549e+06,1.036783e+06,1.148878e+06


In [8]:
#Approach:
#Make a temp df to store in values from a list
#Append temp df to output df

target_columns = ["Amount", "BeneficialUseCV", "ReportYearCV", "STATE", "COUNTY", "WaterSource"]
dfout = pd.DataFrame(columns=target_columns)
dftemp = pd.DataFrame(columns=target_columns)

# for row in df.index:
for row in df.index:
    #Get Values
    Amounts = df.iloc[row].drop(["USE TYPE", "STATE", "COUNTY"]).values.tolist()
    BenUse = df.iloc[row,0]
    ReportYears = df.iloc[0].drop(["USE TYPE", "STATE", "COUNTY"]).index.values.tolist()
    State = str(df.iloc[row,1])
    
    CWS = str(df.iloc[row,2])
    if '-' in CWS:
        County = str(df.iloc[row,2].split('-')[0])
        WaterSource = str(df.iloc[row,2].split('-')[1])
    else:
        County = ""
        WaterSource = str(df.iloc[row,2])  
    
    #Put Values into temp Dataframe
    dftemp['Amount'] = Amounts
    dftemp['ReportYearCV'] = ReportYears
    dftemp['BeneficialUseCV'] = BenUse
    dftemp['STATE'] = State
    dftemp['COUNTY'] = County
    dftemp['WaterSource'] = WaterSource
    
    #Append to dfout
    dfout = dfout.append(dftemp)

dfout
dfout.reset_index()

,index,Amount,BeneficialUseCV,ReportYearCV,STATE,COUNTY,WaterSource
0,0,2500.0,Irrigated Agriculture,1971,Arizona,san juan,colorado river
1,1,2900.0,Irrigated Agriculture,1972,Arizona,san juan,colorado river
2,2,4000.0,Irrigated Agriculture,1973,Arizona,san juan,colorado river
3,3,4250.0,Irrigated Agriculture,1974,Arizona,san juan,colorado river
4,4,3540.0,Irrigated Agriculture,1975,Arizona,san juan,colorado river
...,...,...,...,...,...,...,...
3883,43,0.0,Fish & Wildlife,2014,Wyoming,,green river
3884,44,0.0,Fish & Wildlife,2015,Wyoming,,green river
3885,45,0.0,Fish & Wildlife,2016,Wyoming,,green river
3886,46,0.0,Fish & Wildlife,2017,Wyoming,,green river


In [9]:
# Convert State to abbreveated version
StateAbDic = {
"Arizona" : "AZ",
"Colorado" : "CO",
"New Mexico" : "NM",
"Utah" : "UT",
"Wyoming" : "WY"}

def assignStateAb(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ""
    else:
        String1 = colrowValue  # remove whitespace chars
        try:
            outList = StateAbDic[String1]
        except:
            outList = ""
    return outList

dfout['STATE'] = dfout.apply(lambda row: assignStateAb(row['STATE']), axis=1)
dfout

,Amount,BeneficialUseCV,ReportYearCV,STATE,COUNTY,WaterSource
0,2500.0,Irrigated Agriculture,1971,AZ,san juan,colorado river
1,2900.0,Irrigated Agriculture,1972,AZ,san juan,colorado river
2,4000.0,Irrigated Agriculture,1973,AZ,san juan,colorado river
3,4250.0,Irrigated Agriculture,1974,AZ,san juan,colorado river
4,3540.0,Irrigated Agriculture,1975,AZ,san juan,colorado river
...,...,...,...,...,...,...
43,0.0,Fish & Wildlife,2014,WY,,green river
44,0.0,Fish & Wildlife,2015,WY,,green river
45,0.0,Fish & Wildlife,2016,WY,,green river
46,0.0,Fish & Wildlife,2017,WY,,green river


In [10]:
# Creating ReportingUnitName field.

dfout['in_ReportingUnitName'] = dfout['STATE'] + "_" + dfout['WaterSource']
dfout

,Amount,BeneficialUseCV,ReportYearCV,STATE,COUNTY,WaterSource,in_ReportingUnitName
0,2500.0,Irrigated Agriculture,1971,AZ,san juan,colorado river,AZ_colorado river
1,2900.0,Irrigated Agriculture,1972,AZ,san juan,colorado river,AZ_colorado river
2,4000.0,Irrigated Agriculture,1973,AZ,san juan,colorado river,AZ_colorado river
3,4250.0,Irrigated Agriculture,1974,AZ,san juan,colorado river,AZ_colorado river
4,3540.0,Irrigated Agriculture,1975,AZ,san juan,colorado river,AZ_colorado river
...,...,...,...,...,...,...,...
43,0.0,Fish & Wildlife,2014,WY,,green river,WY_green river
44,0.0,Fish & Wildlife,2015,WY,,green river,WY_green river
45,0.0,Fish & Wildlife,2016,WY,,green river,WY_green river
46,0.0,Fish & Wildlife,2017,WY,,green river,WY_green river


In [11]:
#Fixing Datatypes
dfout['Amount'] = dfout['Amount'].astype(float) 
dfout['ReportYearCV'] = dfout['ReportYearCV'].astype(int)
dfout.dtypes

Amount                  float64
BeneficialUseCV          object
ReportYearCV              int32
STATE                    object
COUNTY                   object
WaterSource              object
in_ReportingUnitName     object
dtype: object

In [12]:
# TimeframeStart & TimeframeEnd

dfout['in_TimeframeStart'] = '01/01/' + dfout['ReportYearCV'].astype(str)
dfout['in_TimeframeEnd'] = '12/31/' + dfout['ReportYearCV'].astype(str)
dfout.head(3)

,Amount,BeneficialUseCV,ReportYearCV,STATE,COUNTY,WaterSource,in_ReportingUnitName,in_TimeframeStart,in_TimeframeEnd
0,2500.0,Irrigated Agriculture,1971,AZ,san juan,colorado river,AZ_colorado river,01/01/1971,12/31/1971
1,2900.0,Irrigated Agriculture,1972,AZ,san juan,colorado river,AZ_colorado river,01/01/1972,12/31/1972
2,4000.0,Irrigated Agriculture,1973,AZ,san juan,colorado river,AZ_colorado river,01/01/1973,12/31/1973


## WaDE Custom Elements (due to missing reporting unit and water source info)

In [13]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEUSBR_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceType'] = dfout['WaterSource']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceType'] == A), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfout['in_WaterSourceNativeID'] = dfout.apply(lambda row: retrieveWaterSourceNativeID(row['WaterSource']), axis=1)
dfout

,Amount,BeneficialUseCV,ReportYearCV,STATE,COUNTY,WaterSource,in_ReportingUnitName,in_TimeframeStart,in_TimeframeEnd,in_WaterSourceNativeID
0,2500.0,Irrigated Agriculture,1971,AZ,san juan,colorado river,AZ_colorado river,01/01/1971,12/31/1971,WaDEUSBR_WS1
1,2900.0,Irrigated Agriculture,1972,AZ,san juan,colorado river,AZ_colorado river,01/01/1972,12/31/1972,WaDEUSBR_WS1
2,4000.0,Irrigated Agriculture,1973,AZ,san juan,colorado river,AZ_colorado river,01/01/1973,12/31/1973,WaDEUSBR_WS1
3,4250.0,Irrigated Agriculture,1974,AZ,san juan,colorado river,AZ_colorado river,01/01/1974,12/31/1974,WaDEUSBR_WS1
4,3540.0,Irrigated Agriculture,1975,AZ,san juan,colorado river,AZ_colorado river,01/01/1975,12/31/1975,WaDEUSBR_WS1
...,...,...,...,...,...,...,...,...,...,...
43,0.0,Fish & Wildlife,2014,WY,,green river,WY_green river,01/01/2014,12/31/2014,WaDEUSBR_WS2
44,0.0,Fish & Wildlife,2015,WY,,green river,WY_green river,01/01/2015,12/31/2015,WaDEUSBR_WS2
45,0.0,Fish & Wildlife,2016,WY,,green river,WY_green river,01/01/2016,12/31/2016,WaDEUSBR_WS2
46,0.0,Fish & Wildlife,2017,WY,,green river,WY_green river,01/01/2017,12/31/2017,WaDEUSBR_WS2


In [14]:
# Creating WaDE Custom reportingunit native ID for easy site identificaiion
# ----------------------------------------------------------------------------------------------------

# Create temp ReportingUnitNativeID dataframe of unique reporting units.
def assignReportingUnitNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEUSBR_RU" + string1
    return outstring

dfReportingUnitNativeID = pd.DataFrame()
dfReportingUnitNativeID['in_ReportingUnitName'] = dfout['in_ReportingUnitName']
dfReportingUnitNativeID = dfReportingUnitNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfReportingUnitNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfReportingUnitNativeID['in_ReportingUnitNativeID'] = dftemp.apply(lambda row: assignReportingUnitNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom reportingunit native ID
def retrieveReportingUnitNativeID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfReportingUnitNativeID.loc[(dfReportingUnitNativeID['in_ReportingUnitName'] == A), 'in_ReportingUnitNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfout['in_ReportingUnitNativeID'] = dfout.apply(lambda row: retrieveReportingUnitNativeID( row['in_ReportingUnitName']), axis=1)
dfout

,Amount,BeneficialUseCV,ReportYearCV,STATE,COUNTY,WaterSource,in_ReportingUnitName,in_TimeframeStart,in_TimeframeEnd,in_WaterSourceNativeID,in_ReportingUnitNativeID
0,2500.0,Irrigated Agriculture,1971,AZ,san juan,colorado river,AZ_colorado river,01/01/1971,12/31/1971,WaDEUSBR_WS1,WaDEUSBR_RU1
1,2900.0,Irrigated Agriculture,1972,AZ,san juan,colorado river,AZ_colorado river,01/01/1972,12/31/1972,WaDEUSBR_WS1,WaDEUSBR_RU1
2,4000.0,Irrigated Agriculture,1973,AZ,san juan,colorado river,AZ_colorado river,01/01/1973,12/31/1973,WaDEUSBR_WS1,WaDEUSBR_RU1
3,4250.0,Irrigated Agriculture,1974,AZ,san juan,colorado river,AZ_colorado river,01/01/1974,12/31/1974,WaDEUSBR_WS1,WaDEUSBR_RU1
4,3540.0,Irrigated Agriculture,1975,AZ,san juan,colorado river,AZ_colorado river,01/01/1975,12/31/1975,WaDEUSBR_WS1,WaDEUSBR_RU1
...,...,...,...,...,...,...,...,...,...,...,...
43,0.0,Fish & Wildlife,2014,WY,,green river,WY_green river,01/01/2014,12/31/2014,WaDEUSBR_WS2,WaDEUSBR_RU9
44,0.0,Fish & Wildlife,2015,WY,,green river,WY_green river,01/01/2015,12/31/2015,WaDEUSBR_WS2,WaDEUSBR_RU9
45,0.0,Fish & Wildlife,2016,WY,,green river,WY_green river,01/01/2016,12/31/2016,WaDEUSBR_WS2,WaDEUSBR_RU9
46,0.0,Fish & Wildlife,2017,WY,,green river,WY_green river,01/01/2017,12/31/2017,WaDEUSBR_WS2,WaDEUSBR_RU9


# Shapefile Data (to crate WKT geometry input)

In [15]:
#Shapefile input
shfile = gpd.read_file('C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/USBR/Colorado River Basin Natural Flow and Salt Data/Summary Sheet/AggregatedAmounts/RawInputData/byState/ByState.shp')
dfshape = pd.DataFrame(shfile)
dfshape.head(3)

,OBJECTID,TNMID,MetaSource,SourceData,SourceOrig,SourceFeat,LoadDate,GNIS_ID,AreaAcres,AreaSqKm,States,HUC10,Name,HUType,HUMod,Shape_Leng,Shape_Area,geometry
0,806,{2FF08A58-1AE2-4B3A-9E60-69AFDD50E9B8},{85FD5379-677D-43E4-B5EF-E9E7E7297246},None,None,None,2012-10-24,0,207648.00,840.32,CO,1404010703,green river,S,None,2.294248,0.090046,"POLYGON ((-109.04826 39.50320, -109.05007 40.9..."
1,140,{C08F01A7-9AF5-4672-8FC5-3E918095EC10},None,None,None,None,2017-09-19,0,15955412.63,64569.32,AZ,None,colorado river,None,None,18.365110,6.520576,"POLYGON ((-109.04522 36.99735, -109.04604 36.0..."
2,38,{30AA0342-4EDA-487D-90D0-DE39A38E1313},None,None,None,None,2017-04-17,0,6296353.12,25480.46,CO,None,upper main stem,None,None,13.295278,2.671646,"POLYGON ((-109.04634 37.92188, -109.04826 39.5..."


In [16]:
#Create output shape dataframe
dfshapeout = pd.DataFrame(columns = ['STATE', "geometry"])
dfshapeout['STATE'] = dfshape['States']
dfshapeout['geometry'] = dfshape['geometry']
dfshapeout

,STATE,geometry
0,CO,"POLYGON ((-109.04826 39.50320, -109.05007 40.9..."
1,AZ,"POLYGON ((-109.04522 36.99735, -109.04604 36.0..."
2,CO,"POLYGON ((-109.04634 37.92188, -109.04826 39.5..."
3,WY,"POLYGON ((-106.86125 41.00002, -110.72029 40.9..."
4,UT,"POLYGON ((-109.05007 40.99894, -109.04826 39.5..."
5,UT,"POLYGON ((-109.04826 39.50320, -109.04634 37.9..."
6,UT,"POLYGON ((-109.05007 37.92160, -109.05006 36.9..."
7,NM,"POLYGON ((-109.04604 36.03339, -109.04522 36.9..."
8,CO,"POLYGON ((-109.05006 36.99735, -109.05007 37.9..."


# Inspect Output Data & Export

In [17]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfout.dtypes)

Amount                      float64
BeneficialUseCV              object
ReportYearCV                  int32
STATE                        object
COUNTY                       object
WaterSource                  object
in_ReportingUnitName         object
in_TimeframeStart            object
in_TimeframeEnd              object
in_WaterSourceNativeID       object
in_ReportingUnitNativeID     object
dtype: object


In [18]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfshapeout.dtypes)

STATE         object
geometry    geometry
dtype: object


In [19]:
# dfout out to CSV.
dfout.to_csv('P_usbrAggMaster.csv', index=False) # The output.
dfshapeout.to_csv('P_usbrGeometry.csv', index=False) # The output geometry.